In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install surprise
# Surprise is an easy-to-use Python library that allows us to quickly build rating-based
# recommender systems without reinventing the wheel.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357289 sha256=18b29da65dcbfcb10de4eb17009e3cc927f1cb09e3c29161d44d802dec6f4c9f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from surprise import Dataset, Reader, KNNWithMeans,SVD, SVDpp,NMF, BaselineOnly, accuracy
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from collections import defaultdict

In [ ]:
movies        = pd.read_csv('/content/drive/MyDrive/MSc. Dissertation/movies.csv')
links         = pd.read_csv('/content/drive/MyDrive/MSc. Dissertation/links.csv')
ratings       = pd.read_csv('/content/drive/MyDrive/MSc. Dissertation/ratings.csv')
tags          = pd.read_csv('/content/drive/MyDrive/MSc. Dissertation/tags.csv')

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [ ]:
movies.isnull().sum()

,0
movieId,0
title,0
genres,0


In [ ]:
print(movies['genres'].nunique())
print(movies['genres'].unique())

951
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Drama' 'Action|Thriller'
 'Drama|Horror|Thriller' 'Comedy|Horror|Roman

In [ ]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [ ]:
links.isnull().sum() # There is some missing values in links dataframe

,0
movieId,0
imdbId,0
tmdbId,8


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [ ]:
ratings['userId'].nunique()

610

In [ ]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [ ]:
tags.isnull().sum()

,0
userId,0
movieId,0
tag,0
timestamp,0


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

# Movie Recommendation : KNNWithMeans

### KNNWithMeans improves upon the standard kNN algorithm by considering the user's average rating when making predictions. When predicting a user's rating for an item, the algorithm computes the weighted average of the ratings given by similar users but adjusts for the users' individual rating biases.

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.20)

In [ ]:
# Initialize and train the KNNWithMeans model
model = KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True})
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"KNNWithMeans training completed in {fit_time:.2f} seconds.")

Computing the cosine similarity matrix...
Done computing similarity matrix.
KNNWithMeans training completed in 0.56 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)
predictions

[Prediction(uid=82, iid=442, r_ui=3.5, est=3.0922120512302844, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=230, iid=6365, r_ui=4.0, est=2.8670954989031316, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=43, iid=161, r_ui=4.0, est=4.878028584744475, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=267, iid=2861, r_ui=2.0, est=4.115273496443002, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=307, iid=8907, r_ui=2.5, est=1.6802109080243834, details={'actual_k': 11, 'was_impossible': False}),
 Prediction(uid=448, iid=3327, r_ui=4.0, est=3.145361630174855, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid=37, iid=231, r_ui=5.0, est=3.713006881642636, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=16, iid=4226, r_ui=4.5, est=4.29154701467539, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=41, iid=49530, r_ui=4.0, est=3.459249641175786, details={'actua

In [ ]:
# Calculate MSE
mse = accuracy.mse(predictions, verbose=False)

# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = accuracy.mae(predictions, verbose=False)

# Print the results
print(f"Training Time: {fit_time:.2f} seconds")
print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

Training Time: 0.56 seconds
Test MSE: 0.8226
Test RMSE: 0.9070
Test MAE: 0.6954


MSE - Mean Square Error,
RMSE - Root Mean Square Error,
MAE - Mean Absolute Error

In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some Movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some Movies recommendations for User 1:
Movie ID: 1090, Estimated Rating: 5.00
Movie ID: 1208, Estimated Rating: 4.88
Movie ID: 3729, Estimated Rating: 4.82
Movie ID: 3053, Estimated Rating: 4.81
Movie ID: 1954, Estimated Rating: 4.74
Movie ID: 943, Estimated Rating: 4.74
Movie ID: 2033, Estimated Rating: 4.73
Movie ID: 1291, Estimated Rating: 4.73
Movie ID: 919, Estimated Rating: 4.72
Movie ID: 1089, Estimated Rating: 4.71


In [ ]:
from collections import defaultdict

# Function to count predictions for each user ID
def count_predictions(predictions):
    prediction_counts = defaultdict(int)
    for prediction in predictions:
        prediction_counts[prediction.uid] += 1
    return prediction_counts

# Generate predictions if not already done
# Ensure you have a predictions list from model.test(testset) or similar

# Count predictions for each user
prediction_counts = count_predictions(predictions)

# Print the number of predictions for each user ID
for user_id, count in prediction_counts.items():
    print(f"User ID {user_id} has {count} predictions.")

# You might want to print the counts for specific users, such as user 1 or 2
print(f"Number of predictions for user 1: {prediction_counts[1]}")
if 2 in prediction_counts:
    print(f"Number of predictions for user 2: {prediction_counts[2]}")
else:
    print("No predictions found for user 2.")


User ID 82 has 49 predictions.
User ID 230 has 30 predictions.
User ID 43 has 23 predictions.
User ID 267 has 15 predictions.
User ID 307 has 187 predictions.
User ID 448 has 371 predictions.
User ID 37 has 3 predictions.
User ID 16 has 21 predictions.
User ID 41 has 36 predictions.
User ID 582 has 11 predictions.
User ID 305 has 133 predictions.
User ID 274 has 242 predictions.
User ID 162 has 10 predictions.
User ID 140 has 109 predictions.
User ID 361 has 26 predictions.
User ID 219 has 103 predictions.
User ID 182 has 205 predictions.
User ID 68 has 245 predictions.
User ID 211 has 16 predictions.
User ID 526 has 13 predictions.
User ID 414 has 571 predictions.
User ID 217 has 123 predictions.
User ID 84 has 59 predictions.
User ID 222 has 50 predictions.
User ID 435 has 11 predictions.
User ID 298 has 203 predictions.
User ID 123 has 10 predictions.
User ID 474 has 432 predictions.
User ID 599 has 516 predictions.
User ID 61 has 10 predictions.
User ID 382 has 61 predictions.
User